<a href="https://colab.research.google.com/github/shivangibithel/IRMiDis_Task2/blob/main/Text_Classification_Methods_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[A Comprehensive Guide to Understand and Implement Text Classification in Python](https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/)

Models
1. Naive Bayes Classifier
2. Linear Classifier
3. Support Vector Machine
4. Bagging Models
5. Boosting Models
6. Shallow Neural Networks
7. Deep Neural Networks
    1. Convolutional Neural Network (CNN)
    2. Long Short Term Modelr (LSTM)
    3. Gated Recurrent Unit (GRU)
    4. Bidirectional RNN
    5. Recurrent Convolutional Neural Network (RCNN)
    Other Variants of Deep Neural Networks

# Imports

In [2]:
# pip install emoji --upgrade

In [3]:
import numpy as np
import pandas as pd #to work with csv files

#matplotlib imports are used to plot confusion matrices for the classifiers
import matplotlib as mpl 
import matplotlib.cm as cm 
import matplotlib.pyplot as plt 

#import feature extraction methods from sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import stop_words

#pre-processing of text
import string
import re

#import classifiers from sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

#import different metrics to evaluate the classifiers
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn import metrics

#import time function from time module to track the training duration
from time import time

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
import sys
import re
!pip install emoji --quiet
import emoji
!pip install contractions --quiet
import contractions
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import unicodedata


     |████████████████████████████████| 133kB 31.1MB/s 
     |████████████████████████████████| 286kB 35.8MB/s 
     |████████████████████████████████| 327kB 46.9MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Section 1: Load and explore the dataset

In [86]:
# our_data = pd.read_csv("irmidis-2021-task2-train.csv")
# test_data = pd.read_csv("irmidis-2021-task2-test.csv")
our_data = pd.read_csv("cleaned_dataset_train.csv",index_col = "id")
test_data = pd.read_csv("cleaned_dataset_test.csv",index_col = "id")
# our_data = shuffle(our_data)
our_data.head()

,Unnamed: 0,tweet,label
id,,,
1325682517148569600,0,coronavirus some canadians hesitant to take a...,AntiVax
1325684180483600384,1,moderna on track to report late stage covid 19...,Neutral
1325689685943996416,2,the philippine government is in talks with ang...,Neutral
1325690517724782593,3,care homes to be first to receive coronavirus ...,Neutral
1325697646909132800,4,this is why thinking that a vaccine is a panac...,AntiVax


In [7]:
display(our_data.shape) #Number of rows (instances) and columns in the dataset
print(our_data["label"].value_counts()/our_data.shape[0]) #Class distribution in the dataset

(2792, 3)

Neutral    0.361748
ProVax     0.354943
AntiVax    0.283309
Name: label, dtype: float64


In [8]:
# Prepare Dataset
trainDF = pd.DataFrame()
trainDF['tweet'] = our_data.tweet
trainDF['label'] = our_data.label

# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['tweet'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
# train_y = encoder.fit_transform(trainDF['label'])
valid_y = encoder.fit_transform(valid_y)

In [87]:
trainDF

,tweet,label
id,,
1325682517148569600,coronavirus some canadians hesitant to take a...,AntiVax
1325684180483600384,moderna on track to report late stage covid 19...,Neutral
1325689685943996416,the philippine government is in talks with ang...,Neutral
1325690517724782593,care homes to be first to receive coronavirus ...,Neutral
1325697646909132800,this is why thinking that a vaccine is a panac...,AntiVax
...,...,...
1325972470336278530,if a covid 19 vaccine is effective it will be ...,Neutral
1325770248382001152,pfizer claims coronavirus vaccine success pla...,Neutral
1325784714062409728,pump dat covid19 vaccine flexed biceps face...,ProVax


### Section 2: Text Pre-processing

Typical steps involve tokenization, lower casing, removing, stop words, punctuation markers etc, and vectorization. Other processes such as stemming/lemmatization can also be performed. Here, we are performing the following steps: removing br tags, punctuation, numbers, and stopwords. While we are using sklearn's list of stopwords, there are several other stop word lists (e.g., from NLTK) or sometimes, custom stopword lists are needed depending on the task. 

In [65]:
# def cleaning(text):
#   text= text.lower()
#   text= emoji.demojize(text)
#   text=contractions.fix(text)
#   text=text.strip()
#   text=text.replace('[^\w\s]','')
#   text=re.sub(r'http\S+', '', text)
#   REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
#   BAD_SYMBOLS_RE = re.compile('[^0-9a-z +]')
#   text = REPLACE_BY_SPACE_RE.sub(' ' , text)
#   text = BAD_SYMBOLS_RE.sub(' ',text)
  
#   return text

# clean=trainDF['tweet'].apply(cleaning)
# # STOPWORDS = set(stopwords.words('english'))

# ff=[]
# for i in clean:
#   text=unicodedata.normalize('NFKD', i).encode('ascii', 'ignore').decode('utf-8', 'ignore')
#   ff.append(text)
# dd=pd.DataFrame(ff)
# dataset = pd.concat([trainDF['id'],dd,trainDF['label']],axis=1)
# dataset.columns = ["id","tweet","label"]
# dataset

,id,tweet,label
0,1325682517148569600,coronavirus some canadians hesitant to take a...,AntiVax
1,1325684180483600384,moderna on track to report late stage covid 19...,Neutral
2,1325689685943996416,the philippine government is in talks with ang...,Neutral
3,1325690517724782593,care homes to be first to receive coronavirus ...,Neutral
4,1325697646909132800,this is why thinking that a vaccine is a panac...,AntiVax
...,...,...,...
2787,1325972470336278530,if a covid 19 vaccine is effective it will be ...,Neutral
2788,1325770248382001152,pfizer claims coronavirus vaccine success pla...,Neutral
2789,1325784714062409728,pump dat covid19 vaccine flexed biceps face...,ProVax
2790,1325992175218417665,lol not a chance i would get any vaccine let ...,AntiVax


In [66]:
# clean_test=test_data['tweet'].apply(cleaning)
# ff=[]
# for i in clean_test:
#   text=unicodedata.normalize('NFKD', i).encode('ascii', 'ignore').decode('utf-8', 'ignore')
#   ff.append(text)
# dd=pd.DataFrame(ff)
# dataset_test = pd.concat([test_data['id'],dd],axis=1)
# dataset_test.columns = ["id","tweet"]
# dataset_test

,id,tweet
0,1259097719341297664,polls say millions of americans will refuse co...
1,1307596484251062273,you are right one should also remember that t...
2,1296241188823953409,so i should have got the keys to my new ho...
3,1294639481216045056,cdc july 2020 acip meeting overview of cov...
4,1319741954721239041,notably absent is a vaccine for the nightmare ...
...,...,...
1595,1275394676594540547,unlocked face with medical mask syringe vi...
1596,1339072387363508224,aakobel ethanhovermale dolmenlord strictly...
1597,1328165880660955142,charliekirk11 it is been 9 months since covid...
1598,1337209455986020353,apparently the covid 19 vaccine caused bell s ...


In [67]:
# trainDF = dataset
# trainDF

,id,tweet,label
0,1325682517148569600,coronavirus some canadians hesitant to take a...,AntiVax
1,1325684180483600384,moderna on track to report late stage covid 19...,Neutral
2,1325689685943996416,the philippine government is in talks with ang...,Neutral
3,1325690517724782593,care homes to be first to receive coronavirus ...,Neutral
4,1325697646909132800,this is why thinking that a vaccine is a panac...,AntiVax
...,...,...,...
2787,1325972470336278530,if a covid 19 vaccine is effective it will be ...,Neutral
2788,1325770248382001152,pfizer claims coronavirus vaccine success pla...,Neutral
2789,1325784714062409728,pump dat covid19 vaccine flexed biceps face...,ProVax
2790,1325992175218417665,lol not a chance i would get any vaccine let ...,AntiVax


In [81]:
# trainDF.to_csv("cleaned_dataset_train.csv")
# dataset_test.to_csv("cleaned_dataset_test.csv")

### commented

In [69]:
# stopwords = stop_words.ENGLISH_STOP_WORDS
# def clean(doc): #doc is a string of text
#     # doc = doc.replace("@", " ")
#     # doc = doc.replace(" ", " ")
#     doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
#     # doc = " ".join([token for token in doc.split() if token not in stopwords])
#     #remove punctuation and numbers
#     return doc

In [70]:
# import re
# def remove_emojis(data):
#     emoj = re.compile("["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#         u"\U00002500-\U00002BEF"  # chinese char
#         u"\U00002702-\U000027B0"
#         u"\U00002702-\U000027B0"
#         u"\U000024C2-\U0001F251"
#         u"\U0001f926-\U0001f937"
#         u"\U00010000-\U0010ffff"
#         u"\u2640-\u2642" 
#         u"\u2600-\u2B55"
#         u"\u200d"
#         u"\u23cf"
#         u"\u23e9"
#         u"\u231a"
#         u"\ufe0f"  # dingbats
#         u"\u3030"
#                       "]+", re.UNICODE)
#     return re.sub(emoj, '', data)

In [71]:
# def remove_url(data):
#   url = re.compile("http[s]?\:\/\/.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+", re.UNICODE)
#   return re.sub(url, '', data)

In [72]:
# for i in range(len(our_data)):
#   our_data.tweet.iloc[i] = remove_url(our_data.tweet.iloc[i])
#   our_data.tweet.iloc[i] = clean(our_data.tweet.iloc[i])
#   our_data.tweet.iloc[i] = remove_emojis(our_data.tweet.iloc[i])

In [73]:
# our_data.head()

# Section 3: Modeling

Now we are ready for the modelling. We are going to use algorithms from sklearn package. We will go through the following steps:

1 Read train data    
2 Extract features from the training data using CountVectorizer, which is a bag of words feature  implementation. We will use the pre-processing function above in conjunction with Count Vectorizer  
3 Transform the test data into the same feature vector as the training data.  
4 Train the classifier  
5 Evaluate the classifier  

# Feature Engineering

In [10]:
X_train = train_x
# X_train = trainDF['tweet']
y_train =train_y
print(X_train.shape, y_train.shape, valid_x.shape, valid_y.shape)

(2094,) (2094,) (698,) (698,)


In [11]:
X_train

id
1335040955490193408    biden urges taking coronavirus vaccine  wearin...
1330276381780025346    sorry to disappoint those retweeting this gloo...
1334847647925088257     albanian government pays 3 9 million dollars ...
1333838050133159937    the government is not proposing to make a covi...
1336113459180744705    well a  tory that talks senses   matthanock  b...
                                             ...                        
1327332997222297600     chelseaclinton  realdonaldtrump who is your d...
1326174223212154880    some good news in addition to the vaccine tria...
1325779532255784961    as a    layman  would someone please explain e...
1325672362029834240    kids are participating in covid 19 vaccine tri...
1325931184342396928    a safe  effective covid 19 vaccination is fina...
Name: tweet, Length: 2094, dtype: object

 ### Count Vectors as features

In [12]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['tweet'])

# transform the training and validation data using count vectorizer object
X_train_dtm =  count_vect.transform(X_train)
xvalid_count =  count_vect.transform(valid_x)

In [13]:
print(X_train_dtm.shape, xvalid_count.shape)

(2094, 7913) (698, 7913)


### TF-IDF Vectors as features

In [18]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['tweet'])
xtrain_tfidf =  tfidf_vect.transform(X_train)
xvalid_tfidf =  tfidf_vect.transform(valid_x)
print(xtrain_tfidf.shape, xvalid_tfidf.shape)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['tweet'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(X_train)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)
print(xtrain_tfidf_ngram.shape, xvalid_tfidf_ngram.shape)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['tweet'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_train)
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 
print(xtrain_tfidf_ngram_chars.shape, xvalid_tfidf_ngram_chars.shape)

(2094, 5000) (698, 5000)
(2094, 5000) (698, 5000)
(2094, 5000) (698, 5000)


In [19]:
print(xtrain_tfidf_ngram)

  (0, 4717)	0.4337950763867642
  (0, 3921)	0.4337950763867642
  (0, 3324)	0.4057409908675666
  (0, 3323)	0.4003403161927118
  (0, 2623)	0.2874276917197443
  (0, 1002)	0.2030829524926654
  (0, 727)	0.418196340736321
  (1, 4354)	0.1574970934766017
  (1, 3351)	0.36260663635478096
  (1, 2553)	0.45132837651731544
  (1, 2552)	0.45132837651731544
  (1, 2517)	0.19099285644584413
  (1, 2516)	0.16460603118893888
  (1, 2331)	0.21737968170274938
  (1, 2213)	0.16134245543534126
  (1, 2199)	0.2077444503941093
  (1, 2116)	0.1255614528344626
  (1, 2080)	0.21737968170274938
  (1, 1805)	0.16347657514575695
  (1, 1652)	0.10712461965069046
  (1, 1055)	0.07038520064016571
  (1, 1042)	0.0586361052446266
  (1, 300)	0.21095371918789332
  (1, 299)	0.17250910915282597
  (1, 202)	0.12116475104186661
  :	:
  (2093, 4556)	0.16946510934012424
  (2093, 4311)	0.24500039914662508
  (2093, 3927)	0.21216737374153435
  (2093, 3926)	0.17193134596114057
  (2093, 3773)	0.23775794038180914
  (2093, 3772)	0.20674538277409915


### Word Embeddings  --> Not Working

In [68]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [69]:
%cd gdrive/MyDrive/

/content/gdrive/MyDrive


Can also use pre-train embeddings from Gensim library in future

In [72]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['tweet'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)
# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

### Text / NLP based features

In [ ]:
trainDF = our_data
trainDF['char_count'] = trainDF['tweet'].apply(len)
trainDF['word_count'] = trainDF['tweet'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['tweet'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['tweet'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['tweet'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [ ]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt
trainDF['noun_count'] = trainDF['tweet'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDF['verb_count'] = trainDF['tweet'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDF['adj_count'] = trainDF['tweet'].apply(lambda x: check_pos_tag(x, 'adj'))
trainDF['adv_count'] = trainDF['tweet'].apply(lambda x: check_pos_tag(x, 'adv'))
trainDF['pron_count'] = trainDF['tweet'].apply(lambda x: check_pos_tag(x, 'pron'))

In [ ]:
trainDF

### Topic Models as features

In [ ]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(X_train_dtm)
topic_word = lda_model.components_ 
vocab = vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [ ]:
topic_summaries

# Classifiers

### Non Deep Methods

In [24]:
# Naive Bayes Classifier
nb = MultinomialNB() #instantiate a Multinomial Naive Bayes model
# nb.fit(X_train_dtm, y_train)#train the count vectorizer model
nb.fit(xtrain_tfidf, y_train)#train the TF-IDF model
# nb.fit(xtrain_tfidf_ngram, y_train)#train the TF-IDF ngram model
# nb.fit(xtrain_tfidf_ngram_chars, y_train)#train the TF-IDF ngram CHAR model

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [34]:
# Logistic Regression Classifier
logreg = LogisticRegression(class_weight="balanced",max_iter=1000) #instantiate a logistic regression model
# logreg.fit(X_train_dtm, y_train) #fit the model with training data
logreg.fit(xtrain_tfidf, y_train)#train the TF-IDF model
# logreg.fit(xtrain_tfidf_ngram, y_train)#train the TF-IDF ngram model
# logreg.fit(xtrain_tfidf_ngram_chars, y_train)#train the TF-IDF ngram CHAR model

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
# Linear SVM Classifier
classifier = LinearSVC(class_weight='balanced') #instantiate a logistic regression model
# classifier.fit(X_train_dtm, y_train) #fit the model with training data
classifier.fit(xtrain_tfidf, y_train)#train the TF-IDF model
# classifier.fit(xtrain_tfidf_ngram, y_train)#train the TF-IDF ngram model
# classifier.fit(xtrain_tfidf_ngram_chars, y_train)#train the TF-IDF ngram CHAR model

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [60]:
# Bagging Model
classifier = ensemble.RandomForestClassifier()
classifier.fit(X_train_dtm, y_train)
# classifier.fit(xtrain_tfidf, y_train)#train the TF-IDF model
# classifier.fit(xtrain_tfidf_ngram, y_train)#train the TF-IDF ngram model
# classifier.fit(xtrain_tfidf_ngram_chars, y_train)#train the TF-IDF ngram CHAR model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [63]:
# Extereme Gradient Boosting
EXB = xgboost.XGBClassifier()
# EXB.fit(X_train_dtm.tocsc(), y_train)
EXB.fit(xtrain_tfidf.tocsc(), y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [81]:
# Shallow Neural Network
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="tanh")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(3, activation="softmax")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',metrics=["accuracy"])
    return classifier 

classifier = create_model_architecture(70)
classifier.fit(x_train, train_y,batch_size=64, epochs=2, validation_data=(x_val, valid_y))

Epoch 1/2
33/33 [==============================] - 1s 12ms/step - loss: 1.5233 - accuracy: 0.3538 - val_loss: 1.3762 - val_accuracy: 0.3625
Epoch 2/2
33/33 [==============================] - 0s 2ms/step - loss: 1.3155 - accuracy: 0.3966 - val_loss: 1.2990 - val_accuracy: 0.3782


### Deep Neural Models

In [73]:
from tensorflow import keras
maxlen= 70
x_train = keras.preprocessing.sequence.pad_sequences(token.texts_to_sequences(X_train), maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=maxlen)

In [74]:
train_y = keras.utils.to_categorical(train_y, num_classes=3)
valid_y = keras.utils.to_categorical(valid_y, num_classes=3)

In [82]:
# CNN
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="tanh")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="tanh")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)
    
    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',metrics=["accuracy"])
    
    return model

classifier = create_cnn()
classifier.fit(x_train, train_y,batch_size=64, epochs=8, validation_data=(x_val, valid_y))
# classifier.fit(x_train, train_y,batch_size=64, epochs=7)
# train_seq_x -- > Word Embeddings required -->Later
# accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
# print "CNN, Word Embeddings",  accuracy

Epoch 1/8
33/33 [==============================] - 4s 61ms/step - loss: 1.0586 - accuracy: 0.4325 - val_loss: 0.8934 - val_accuracy: 0.5745
Epoch 2/8
33/33 [==============================] - 2s 51ms/step - loss: 0.8557 - accuracy: 0.6196 - val_loss: 0.7543 - val_accuracy: 0.6705
Epoch 3/8
33/33 [==============================] - 2s 51ms/step - loss: 0.7158 - accuracy: 0.7040 - val_loss: 0.6819 - val_accuracy: 0.6905
Epoch 4/8
33/33 [==============================] - 2s 50ms/step - loss: 0.6026 - accuracy: 0.7583 - val_loss: 0.6464 - val_accuracy: 0.7335
Epoch 5/8
33/33 [==============================] - 2s 50ms/step - loss: 0.5598 - accuracy: 0.7639 - val_loss: 0.6011 - val_accuracy: 0.7407
Epoch 6/8
33/33 [==============================] - 2s 51ms/step - loss: 0.4669 - accuracy: 0.8259 - val_loss: 0.5836 - val_accuracy: 0.7464
Epoch 7/8
33/33 [==============================] - 2s 51ms/step - loss: 0.4317 - accuracy: 0.8478 - val_loss: 0.6013 - val_accuracy: 0.7579
Epoch 8/8
33/33 [===

In [ ]:
classifier.summary()

Model: "model_49"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_50 (InputLayer)        [(None, 70)]              0         
_________________________________________________________________
embedding_49 (Embedding)     (None, 70, 300)           2340900   
_________________________________________________________________
spatial_dropout1d_49 (Spatia (None, 70, 300)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 68, 100)           90100     
_________________________________________________________________
global_max_pooling1d_18 (Glo (None, 100)               0         
_________________________________________________________________
dense_98 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_49 (Dropout)         (None, 50)                0  

In [84]:
#  LSTM
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=["accuracy"])
    
    return model

# model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
# model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))

classifier_lstm = create_rnn_lstm()
classifier_lstm.fit(x_train, train_y,batch_size=64, epochs=8, validation_data=(x_val, valid_y))
# train_seq_x -- > Word Embeddings required -->Later

Epoch 1/8
33/33 [==============================] - 9s 199ms/step - loss: 1.0866 - accuracy: 0.3715 - val_loss: 0.9645 - val_accuracy: 0.5215
Epoch 2/8
33/33 [==============================] - 5s 143ms/step - loss: 0.9430 - accuracy: 0.5042 - val_loss: 0.8794 - val_accuracy: 0.5444
Epoch 3/8
33/33 [==============================] - 5s 143ms/step - loss: 0.8377 - accuracy: 0.5962 - val_loss: 0.7945 - val_accuracy: 0.6275
Epoch 4/8
33/33 [==============================] - 5s 145ms/step - loss: 0.7883 - accuracy: 0.6447 - val_loss: 0.7483 - val_accuracy: 0.6705
Epoch 5/8
33/33 [==============================] - 5s 144ms/step - loss: 0.7693 - accuracy: 0.6646 - val_loss: 0.7059 - val_accuracy: 0.6991
Epoch 6/8
33/33 [==============================] - 5s 144ms/step - loss: 0.6496 - accuracy: 0.7365 - val_loss: 0.7271 - val_accuracy: 0.6691
Epoch 7/8
33/33 [==============================] - 5s 144ms/step - loss: 0.6786 - accuracy: 0.7128 - val_loss: 0.7031 - val_accuracy: 0.7092
Epoch 8/8
33/

In [ ]:
classifier_lstm.summary()

In [85]:
def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=["accuracy"])
    
    return model

classifier_rnn = create_rnn_gru()
classifier_rnn.fit(x_train, train_y,batch_size=64, epochs=8, validation_data=(x_val, valid_y))
# train_seq_x -- > Word Embeddings required -->Later

Epoch 1/8
33/33 [==============================] - 8s 138ms/step - loss: 1.0920 - accuracy: 0.3670 - val_loss: 1.0458 - val_accuracy: 0.4355
Epoch 2/8
33/33 [==============================] - 4s 117ms/step - loss: 1.0171 - accuracy: 0.4473 - val_loss: 0.9205 - val_accuracy: 0.5473
Epoch 3/8
33/33 [==============================] - 4s 118ms/step - loss: 0.8942 - accuracy: 0.5525 - val_loss: 0.8334 - val_accuracy: 0.6103
Epoch 4/8
33/33 [==============================] - 4s 119ms/step - loss: 0.8151 - accuracy: 0.6180 - val_loss: 0.8023 - val_accuracy: 0.6103
Epoch 5/8
33/33 [==============================] - 4s 117ms/step - loss: 0.7662 - accuracy: 0.6432 - val_loss: 0.7879 - val_accuracy: 0.6261
Epoch 6/8
33/33 [==============================] - 4s 117ms/step - loss: 0.7687 - accuracy: 0.6455 - val_loss: 0.7347 - val_accuracy: 0.6719
Epoch 7/8
33/33 [==============================] - 4s 117ms/step - loss: 0.6754 - accuracy: 0.7218 - val_loss: 0.7351 - val_accuracy: 0.6648
Epoch 8/8
33/

In [ ]:
def create_bidirectional_rnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="tanh")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=["accuracy"])
    
    return model

classifier_brnn = create_bidirectional_rnn()
classifier_brnn.fit(x_train, train_y,batch_size=64, epochs=8, validation_data=(x_val, valid_y))
# train_seq_x -- > Word Embeddings required -->Later

Epoch 1/8
33/33 [==============================] - 13s 272ms/step - loss: 1.0899 - accuracy: 0.3895 - val_loss: 1.0163 - val_accuracy: 0.4957
Epoch 2/8
33/33 [==============================] - 8s 232ms/step - loss: 0.9697 - accuracy: 0.5343 - val_loss: 0.9229 - val_accuracy: 0.5659
Epoch 3/8
33/33 [==============================] - 8s 230ms/step - loss: 0.8905 - accuracy: 0.5729 - val_loss: 0.8322 - val_accuracy: 0.6089
Epoch 4/8
33/33 [==============================] - 8s 233ms/step - loss: 0.8588 - accuracy: 0.5996 - val_loss: 0.7843 - val_accuracy: 0.6404
Epoch 5/8
33/33 [==============================] - 8s 231ms/step - loss: 0.7753 - accuracy: 0.6518 - val_loss: 0.7320 - val_accuracy: 0.6963
Epoch 6/8
33/33 [==============================] - 7s 228ms/step - loss: 0.7564 - accuracy: 0.6710 - val_loss: 0.7161 - val_accuracy: 0.7020
Epoch 7/8
33/33 [==============================] - 8s 231ms/step - loss: 0.6683 - accuracy: 0.7311 - val_loss: 0.6979 - val_accuracy: 0.7049
Epoch 8/8
33

In [ ]:
def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="tanh")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="tanh")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=["accuracy"])
    
    return model

classifier_rcnn = create_rcnn()
classifier_rcnn.fit(x_train, train_y,batch_size=64, epochs=8, validation_data=(x_val, valid_y))
# train_seq_x -- > Word Embeddings required -->Later

Epoch 1/8
33/33 [==============================] - 3s 65ms/step - loss: 1.1101 - accuracy: 0.3801 - val_loss: 0.9261 - val_accuracy: 0.6003
Epoch 2/8
33/33 [==============================] - 2s 53ms/step - loss: 0.9175 - accuracy: 0.5809 - val_loss: 0.7658 - val_accuracy: 0.6576
Epoch 3/8
33/33 [==============================] - 2s 54ms/step - loss: 0.7363 - accuracy: 0.6994 - val_loss: 0.6849 - val_accuracy: 0.7149
Epoch 4/8
33/33 [==============================] - 2s 54ms/step - loss: 0.6436 - accuracy: 0.7277 - val_loss: 0.6300 - val_accuracy: 0.7450
Epoch 5/8
33/33 [==============================] - 2s 55ms/step - loss: 0.5666 - accuracy: 0.7734 - val_loss: 0.6089 - val_accuracy: 0.7421
Epoch 6/8
33/33 [==============================] - 2s 54ms/step - loss: 0.5035 - accuracy: 0.7911 - val_loss: 0.5885 - val_accuracy: 0.7579
Epoch 7/8
33/33 [==============================] - 2s 54ms/step - loss: 0.4426 - accuracy: 0.8445 - val_loss: 0.5629 - val_accuracy: 0.7808
Epoch 8/8
33/33 [===

# Evaluation

### Naive Bayes Evaluation

In [15]:
predictions = nb.predict(xvalid_count)
print(" NB, Count Vectors:", metrics.accuracy_score(predictions, valid_y))

 NB, Count Vectors: 0.7263610315186246


In [ ]:
predictions

In [21]:
predictions = nb.predict(xvalid_tfidf)
print(" NB, WordLevel TF-IDF:", metrics.accuracy_score(predictions, valid_y))

 NB, WordLevel TF-IDF: 0.7306590257879656


In [23]:
predictions = nb.predict(xvalid_tfidf_ngram)
print(" NB, N-Gram Vectors:", metrics.accuracy_score(predictions, valid_y))

 NB, N-Gram Vectors: 0.7005730659025788


In [25]:
predictions = nb.predict(xvalid_tfidf_ngram_chars)
print(" NB, CharLevel Vectors:", metrics.accuracy_score(predictions, valid_y))

 NB, CharLevel Vectors: 0.6891117478510028


### Linear Classification --> Logistic Regression

In [29]:
predictions = logreg.predict(xvalid_count)
print(" LR, Count Vectors:", metrics.accuracy_score(predictions, valid_y))

 LR, Count Vectors: 0.7378223495702005


In [27]:
predictions = logreg.predict(xvalid_tfidf)
print(" LR, WordLevel TF-IDF:", metrics.accuracy_score(predictions, valid_y))

 LR, WordLevel TF-IDF: 0.7392550143266475


In [31]:
predictions = logreg.predict(xvalid_tfidf_ngram)
print(" LR, N-Gram Vectors:", metrics.accuracy_score(predictions, valid_y))

 LR, N-Gram Vectors: 0.7106017191977078


In [33]:
predictions = logreg.predict(xvalid_tfidf_ngram_chars)
print(" LR, CharLevel Vectors:", metrics.accuracy_score(predictions, valid_y))

 LR, CharLevel Vectors: 0.7249283667621776


### SVM Evaluation

In [41]:
predictions = classifier.predict(xvalid_count)
print(" SVM, Count Vectors:", metrics.accuracy_score(predictions, valid_y))

 SVM, Count Vectors: 0.7048710601719198


In [50]:
predictions = classifier.predict(xvalid_tfidf)
print(" SVM, WordLevel TF-IDF:", metrics.accuracy_score(predictions, valid_y))

 SVM, WordLevel TF-IDF: 0.7335243553008596


In [46]:
predictions = classifier.predict(xvalid_tfidf_ngram)
print(" SVM, N-Gram Vectors:", metrics.accuracy_score(predictions, valid_y))

 SVM, N-Gram Vectors: 0.7034383954154728


In [48]:
predictions = classifier.predict(xvalid_tfidf_ngram_chars)
print(" SVM, CharLevel Vectors:", metrics.accuracy_score(predictions, valid_y))

 SVM, CharLevel Vectors: 0.7091690544412608


### Random Forrest Evaluation Evaluation

In [52]:
predictions = classifier.predict(xvalid_count)
print(" RF, Count Vectors:", metrics.accuracy_score(predictions, valid_y))

 RF, Count Vectors: 0.7148997134670487


In [54]:
predictions = classifier.predict(xvalid_tfidf)
print(" RF, WordLevel TF-IDF:", metrics.accuracy_score(predictions, valid_y))

 RF, WordLevel TF-IDF: 0.7020057306590258


In [57]:
predictions = classifier.predict(xvalid_tfidf_ngram)
print(" RF, N-Gram Vectors:", metrics.accuracy_score(predictions, valid_y))

 RF, N-Gram Vectors: 0.6547277936962751


In [59]:
predictions = classifier.predict(xvalid_tfidf_ngram_chars)
print(" RF, CharLevel Vectors:", metrics.accuracy_score(predictions, valid_y))

 RF, CharLevel Vectors: 0.6833810888252149


### Boosting

In [62]:
predictions = EXB.predict(xvalid_count.tocsc())
print(" EXB, Count Vectors:", metrics.accuracy_score(predictions, valid_y))

 EXB, Count Vectors: 0.6991404011461319


In [64]:
predictions = EXB.predict(xvalid_tfidf.tocsc())
print(" RF, WordLevel TF-IDF:", metrics.accuracy_score(predictions, valid_y))

 RF, WordLevel TF-IDF: 0.7106017191977078


### Shallow NN

In [78]:
predictions = classifier.predict(x_val)
predictions = predictions.argmax(axis=-1)
print(" Shallow NN, WordLevel TF-IDF:", metrics.accuracy_score(predictions, valid_y))

ValueError: ignored

### Deep  --> poor results due to embeddings from different domain
Need better pre-trained embeddings

In [ ]:
# predictions = classifier.predict(x_val)
# predictions = predictions.argmax(axis=-1)
# print(" CNN, WordEmbeddings:", metrics.accuracy_score(predictions, valid_y))
# predictions

In [ ]:
# predictions = classifier_lstm.predict(valid_seq_x)
# predictions = predictions.argmax(axis=-1)
# print(" LSTM, WordEmbeddings:", metrics.accuracy_score(predictions, valid_y))

In [ ]:
# predictions = classifier_rnn.predict(x_val)
# # predictions = predictions.argmax(axis=-1)
# print(" RNN, WordEmbeddings:", metrics.accuracy_score(predictions, valid_y))

In [ ]:
# predictions = classifier_brnn.predict(valid_seq_x)
# predictions = predictions.argmax(axis=-1)
# print(" B RNN, WordEmbeddings:", metrics.accuracy_score(predictions, valid_y))

In [ ]:
# predictions = classifier_rcnn.predict(valid_seq_x)
# predictions = predictions.argmax(axis=-1)
# print(" RCNN, WordEmbeddings:", metrics.accuracy_score(predictions, valid_y))